<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/test_jmespath_deephash_deepdiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -qq jmespath deepdiff python-jsonschema-objects

In [3]:
import jmespath
from deepdiff import DeepDiff
from deepdiff import DeepHash
import python_jsonschema_objects as pjs

In [26]:
deepdiff_options = dict(ignore_order=True, report_repetition=True)
# deephash_options = dict(ignore_iterable_order=True, ignore_repetition=True)

## Demo

In [9]:
example = {
    "title": "Example Schema",
    "type": "object",
    "properties": {
        "firstName": {
            "type": "string"
        },
        "lastName": {
            "type": "string"
        },
        "age": {
            "description": "Age in years",
            "type": "integer",
            "minimum": 0
        },
        "dogs": {
            "type": "array",
            "items": {"type": "string"},
            "maxItems": 4
        },
        "address": {
            "type": "object",
            "properties": {
                "street": {"type": "string"},
                "city": {"type": "string"},
                "state": {"type": "string"}
                },
            "required":["street", "city"]
            },
        "gender": {
            "type": "string",
            "enum": ["male", "female"]
        },
        "deceased": {
            "enum": ["yes", "no", 1, 0, "true", "false"]
            }
    },
    "required": ["firstName", "lastName"]
}

In [11]:
builder = pjs.ObjectBuilder(example)
ns = builder.build_classes()
Person = ns.ExampleSchema

/usr/local/lib/python3.10/dist-packages/python_jsonschema_objects/__init__.py:99: UserWarning: Schema version not specified. Defaulting to http://json-schema.org/draft-04/schema
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/python_jsonschema_objects/__init__.py:112: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [13]:
james = Person(firstName="James", lastName="Bond");james

<example_schema address=None age=None deceased=None dogs=None firstName=<Literal<str> James> gender=None lastName=<Literal<str> Bond>>

In [14]:
nigel_dict = dict(firstName="Nigel", lastName="Bond");nigel_dict

{'firstName': 'Nigel', 'lastName': 'Bond'}

In [15]:
nigel = Person(**nigel_dict);nigel

<example_schema address=None age=None deceased=None dogs=None firstName=<Literal<str> Nigel> gender=None lastName=<Literal<str> Bond>>

## End of Demo


# let's do the schema for one item object

### before json formatter

In [17]:
items = {
    "title": "Item Schema",
    "type": "object",
      "properties":{
        "item_aliases":{
            "type":"array",
            "items":{"type":"string"},
            "maxItems":1
        },
        "item_name":{
            "type":"string"
        },
        "modifications":{
            "type":"array",
            "items":{"type":"string"}
        },
        "quantity":{
            "type":"integer",
            "minimum" : 1
        },
        "special_requests":{
            "type":"array",
            "items":{"type":"string"}
        }
      },
    "required": ["item_name","quantity"]
}

### after json formatter

In [18]:
items = {
   "title":"Item Schema",
   "type":"object",
   "properties":{
      "item_aliases":{
         "type":"array",
         "items":{
            "type":"string"
         },
         "maxItems":1
      },
      "item_name":{
         "type":"string"
      },
      "modifications":{
         "type":"array",
         "items":{
            "type":"string"
         }
      },
      "quantity":{
         "type":"integer",
         "minimum":1
      },
      "special_requests":{
         "type":"array",
         "items":{
            "type":"string"
         }
      }
   },
   "required":[
      "item_name",
      "quantity"
   ]
}

In [20]:
builder = pjs.ObjectBuilder(items)
ns = builder.build_classes()
Item = ns.ItemSchema

/usr/local/lib/python3.10/dist-packages/python_jsonschema_objects/__init__.py:99: UserWarning: Schema version not specified. Defaulting to http://json-schema.org/draft-04/schema
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/python_jsonschema_objects/__init__.py:112: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [21]:
valid_item_json_1 ={
               "item_aliases":[

               ],
               "item_name":"Sourdough Jack",
               "modifications":[
                  "No pickles",
                  "No ketchup"
               ],
               "quantity":1,
               "special_requests":[
               ]
            }

In [23]:
item_1 = Item(**valid_item_json_1);item_1

<item_schema item_aliases=<item_schema/item_aliases_<anonymous_field>=[]> item_name=<Literal<str> Sourdough Jack> modifications=<item_schema/modifications_<anonymous_field>=['No pickles', 'No ketchup']> quantity=<Literal<int> 1> special_requests=<item_schema/special_requests_<anonymous_field>=[]>>

In [22]:
valid_item_json_2 ={
               "item_aliases":[

               ],
               "item_name":"Sourdough Jack",
               "modifications":[

               ],
               "quantity":2,
               "special_requests":[
               ]
            }

In [24]:
item_2 = Item(**valid_item_json_2);item_2

<item_schema item_aliases=<item_schema/item_aliases_<anonymous_field>=[]> item_name=<Literal<str> Sourdough Jack> modifications=<item_schema/modifications_<anonymous_field>=[]> quantity=<Literal<int> 2> special_requests=<item_schema/special_requests_<anonymous_field>=[]>>

In [27]:
DeepDiff(valid_item_json_1, valid_item_json_2, **deepdiff_options)

{'values_changed': {"root['quantity']": {'new_value': 2, 'old_value': 1}},
 'iterable_item_removed': {"root['modifications'][0]": 'No pickles',
  "root['modifications'][1]": 'No ketchup'}}

In [28]:
DeepDiff(item_1, item_2, **deepdiff_options)

{'values_changed': {"root['quantity']._value": {'new_value': 2,
   'old_value': 1}},
 'iterable_item_removed': {"root['modifications'][0]": <Literal<str> No pickles>,
  "root['modifications'][1]": <Literal<str> No ketchup>}}

In [ ]:
valid_category_json ={
   "categories":[
      {
         "items":[
            {
               "item_aliases":[

               ],
               "item_name":"Sourdough Jack",
               "modifications":[
                  "No pickles",
                  "No ketchup"
               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"Sourdough Jack",
               "modifications":[

               ],
               "quantity":2,
               "special_requests":[

               ]
            }
         ],
         "name":"Burger"
      }
   ],
   "combos":[

   ]
}

{'categories': [{'items': [{'item_aliases': [],
     'item_name': 'Sourdough Jack',
     'modifications': ['No pickles', 'No ketchup'],
     'quantity': 1,
     'special_requests': []},
    {'item_aliases': [],
     'item_name': 'Sourdough Jack',
     'modifications': [],
     'quantity': 2,
     'special_requests': []}],
   'name': 'Burger'}],
 'combos': []}

### Let's do the schema for one category object

In [ ]:
category_encoding = {
    "title":"Category Schema",
    "type":"object",
    "properties":{
                "name":{"type":"string", "enum":["Burger", "Salad"]},
                "items":{
                    "type":"array",
                    "properties":{

                    }
                }
            }
        }

In [ ]:
DeepHash(a)

{'categories': '3b0d1d4a7ac46c6b1be649bf9256b3d539abd87dbf3d17bc355b7163a15e4364', 'items': 'd4e2071bc7b6d012f0201cf51c8aa310dc9db296c1f83249c3e8ff7eb4d9e115', 'item_aliases': '4b07fb451f5123031605096b7721290f0a02101a56d5242d8ff57204294a46d8', '!>*id140713540788224': 'e78b481f6a5083ac6d266e434fd0da3dc14bf48ac1376d0476b6e310c721e6d9', 'item_name': '47918fa1360c61a55aa73eba968ca0fcc5391c848ee4b919b5875495f5656a02', 'Sourdough Jack': 'dedbf4fb6e726b488544c1cc2c1544508b0216d3a70b3c51f2ee8ce31107f...}

In [ ]:
two_double_jacks_no_pickles_no_onions = {'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ["no pickles", "no onions"], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []}; two_double_jacks_no_pickles_no_onions

{'categories': [{'items': [{'item_aliases': [],
     'item_name': 'Double Jack',
     'modifications': ['no pickles', 'no onions'],
     'quantity': 2,
     'special_requests': []}],
   'name': 'Burger'}],
 'combos': []}

In [ ]:
two_double_jacks_no_onions_no_pickles = {'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ["no onions", "no pickles"], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []}; two_double_jacks_no_onions_no_pickles

{'categories': [{'items': [{'item_aliases': [],
     'item_name': 'Double Jack',
     'modifications': ['no onions', 'no pickles'],
     'quantity': 2,
     'special_requests': []}],
   'name': 'Burger'}],
 'combos': []}

In [ ]:
DeepHash(two_double_jacks_no_pickles_no_onions) == DeepHash(two_double_jacks_no_onions_no_pickles) #FAIL

False

In [ ]:
DeepHash(two_double_jacks_no_pickles_no_onions)[two_double_jacks_no_pickles_no_onions] == DeepHash(two_double_jacks_no_onions_no_pickles)[two_double_jacks_no_onions_no_pickles] #SUCCEED

True

In [ ]:
DeepDiff(two_double_jacks_no_onions_no_pickles, two_double_jacks_no_pickles_no_onions, **deepdiff_options) #SUCCESS

{}

In [ ]:
# reminder

actual = dict(a=1, b=2, c=3, d=4)
expected = dict(a=1, b=3, c=3)
DeepDiff(actual, expected, **deepdiff_options)


{'dictionary_item_removed': [root['d']],
 'values_changed': {"root['b']": {'new_value': 3, 'old_value': 2}}}

In [ ]:
DeepDiff(expected, actual, **deepdiff_options) # this is the correct order!!!

{'dictionary_item_added': [root['d']],
 'values_changed': {"root['b']": {'new_value': 2, 'old_value': 3}}}

In [ ]:
jmespath.search('categories[*].name',a)

['Burger']

In [ ]:
jmespath.search('categories[*].items',a)

[[{'item_aliases': [],
   'item_name': 'Sourdough Jack',
   'modifications': ['No pickles', 'No ketchup'],
   'quantity': 1,
   'special_requests': []},
  {'item_aliases': [],
   'item_name': 'Sourdough Jack',
   'modifications': [],
   'quantity': 2,
   'special_requests': []}]]

In [ ]:
uccombo = {"categories":[],"combos":[{"combo_aliases":["#6"],"items":[{"item_aliases":[],"item_name":"Ultimate Cheeseburger","modifications":["No ketchup"],"quantity":1,"special_requests":[]},{"item_aliases":[],"item_name":"French Fries Small","modifications":[],"quantity":1,"special_requests":[]},{"item_aliases":[],"item_name":"Small Coca-Cola","modifications":[],"quantity":1,"special_requests":[]}],"name":"Ultimate Cheeseburger Combo"}]};uccombo

{'categories': [],
 'combos': [{'combo_aliases': ['#6'],
   'items': [{'item_aliases': [],
     'item_name': 'Ultimate Cheeseburger',
     'modifications': ['No ketchup'],
     'quantity': 1,
     'special_requests': []},
    {'item_aliases': [],
     'item_name': 'French Fries Small',
     'modifications': [],
     'quantity': 1,
     'special_requests': []},
    {'item_aliases': [],
     'item_name': 'Small Coca-Cola',
     'modifications': [],
     'quantity': 1,
     'special_requests': []}],
   'name': 'Ultimate Cheeseburger Combo'}]}